<a href="https://colab.research.google.com/github/marciogsantana/Ciencia-de-dados/blob/main/arvoreDecisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#noetbook utilizado para a aula sobre árvore de decisão e floresta randômica

In [ ]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [ ]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("ArvoreDecisao").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [ ]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,463094961
dbfs:/FileStore/tables/d1995_07_01-24d0c.json,d1995_07_01-24d0c.json,7451741
dbfs:/FileStore/tables/d1995_07_02-c3f44.json,d1995_07_02-c3f44.json,9572086
dbfs:/FileStore/tables/d1995_07_03-f99af.json,d1995_07_03-f99af.json,14125303
dbfs:/FileStore/tables/d1995_07_04-c5a7f.json,d1995_07_04-c5a7f.json,12030333
dbfs:/FileStore/tables/d1995_07_05-0f261.json,d1995_07_05-0f261.json,14662194
dbfs:/FileStore/tables/d1995_07_06-20619.json,d1995_07_06-20619.json,15557682
dbfs:/FileStore/tables/d1995_07_07-2dd8d.json,d1995_07_07-2dd8d.json,15279295
dbfs:/FileStore/tables/d1995_07_08-83302.json,d1995_07_08-83302.json,7033852
dbfs:/FileStore/tables/d1995_07_09-f75d3.json,d1995_07_09-f75d3.json,5589076


In [ ]:
diretorioArvore="/FileStore/tables/iris_bezdekIris.csv"  #diretório que contém o arquivo a ser utilizado

In [ ]:
#lendo arquivos armazenados CSV com o esquema definido
df_iris = spark.read.format('csv').options(inferSchema=True,header='false',delimiter=',').load(diretorioArvore)

In [ ]:
df_iris.printSchema()

root
-- _c0: double (nullable = true)
-- _c1: double (nullable = true)
-- _c2: double (nullable = true)
-- _c3: double (nullable = true)
-- _c4: string (nullable = true)

In [ ]:
df_iris.show(5)

+---+---+---+---+-----------+
_c0|_c1|_c2|_c3| _c4|
+---+---+---+---+-----------+
5.1|3.5|1.4|0.2|Iris-setosa|
4.9|3.0|1.4|0.2|Iris-setosa|
4.7|3.2|1.3|0.2|Iris-setosa|
4.6|3.1|1.5|0.2|Iris-setosa|
5.0|3.6|1.4|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 5 rows

In [ ]:
#modificando o nome das colunas existentes no cabeçalho 
df_iris = df_iris.selectExpr("_c0 as sep_len", "_c1 as sep_wid", "_c2 as pet_len", "_c3 as pet_wid", "_c4 as label")

In [ ]:
df_iris.show(5)

+-------+-------+-------+-------+-----------+
sep_len|sep_wid|pet_len|pet_wid| label|
+-------+-------+-------+-------+-----------+
 5.1| 3.5| 1.4| 0.2|Iris-setosa|
 4.9| 3.0| 1.4| 0.2|Iris-setosa|
 4.7| 3.2| 1.3| 0.2|Iris-setosa|
 4.6| 3.1| 1.5| 0.2|Iris-setosa|
 5.0| 3.6| 1.4| 0.2|Iris-setosa|
+-------+-------+-------+-------+-----------+
only showing top 5 rows

Conhecendo o Banco de Dados

In [ ]:
#encontrando as "estatísticas"
df_iris.describe(['sep_len','sep_wid','pet_len','pet_wid']).show()

+-------+------------------+-------------------+------------------+------------------+
summary| sep_len| sep_wid| pet_len| pet_wid|
+-------+------------------+-------------------+------------------+------------------+
 count| 150| 150| 150| 150|
 mean| 5.843333333333335| 3.057333333333334|3.7580000000000027| 1.199333333333334|
 stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|
 min| 4.3| 2.0| 1.0| 0.1|
 max| 7.9| 4.4| 6.9| 2.5|
+-------+------------------+-------------------+------------------+------------------+

In [ ]:
#definindo a visão do dataframe para ser utilizado como uma tabela pelo SQL
df_iris.createOrReplaceTempView("irisTable")

In [ ]:
display(spark.sql('select * from irisTable '))

sep_len,sep_wid,pet_len,pet_wid,label
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


Iniciando o Processo de Construção e Aplicação da Árvore de Decisão

In [ ]:
from pyspark.ml.linalg import Vectors  #biblioteca que contém funções de Algebra Linear
from pyspark.ml.feature import VectorAssembler #biblioteca que contém as funções para a construção de vetores

In [ ]:
#criando o vetor de características
vector_assembler = VectorAssembler(inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"],outputCol="features")
df_temp = vector_assembler.transform(df_iris)
df_temp.show(5)

+-------+-------+-------+-------+-----------+-----------------+
sep_len|sep_wid|pet_len|pet_wid| label| features|
+-------+-------+-------+-------+-----------+-----------------+
 5.1| 3.5| 1.4| 0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
 4.9| 3.0| 1.4| 0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
 4.7| 3.2| 1.3| 0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
 4.6| 3.1| 1.5| 0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
 5.0| 3.6| 1.4| 0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 5 rows

In [ ]:
#removendo as colunas que não serão utilizadas
df_menor = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df_menor.show(5)

+-----------+-----------------+
 label| features|
+-----------+-----------------+
Iris-setosa|[5.1,3.5,1.4,0.2]|
Iris-setosa|[4.9,3.0,1.4,0.2]|
Iris-setosa|[4.7,3.2,1.3,0.2]|
Iris-setosa|[4.6,3.1,1.5,0.2]|
Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+-----------------+
only showing top 5 rows

In [ ]:
#aplicando as transformações para a coluna label
from pyspark.ml.feature import StringIndexer  #cria o 'vetor' para cada uma das classes existentes na coluna label

l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")  #cria o objeto para a codificação
df_final = l_indexer.fit(df_menor).transform(df_menor)  #aplica a transformação

In [ ]:
df_final.show(5)

+-----------+-----------------+----------+
 label| features|labelIndex|
+-----------+-----------------+----------+
Iris-setosa|[5.1,3.5,1.4,0.2]| 0.0|
Iris-setosa|[4.9,3.0,1.4,0.2]| 0.0|
Iris-setosa|[4.7,3.2,1.3,0.2]| 0.0|
Iris-setosa|[4.6,3.1,1.5,0.2]| 0.0|
Iris-setosa|[5.0,3.6,1.4,0.2]| 0.0|
+-----------+-----------------+----------+
only showing top 5 rows

In [ ]:
#dividindo entre dados de treinamento e teste
(train, test) = df_final.randomSplit([0.7, 0.3])

In [ ]:
test.show(5)

+-----------+-----------------+----------+
 label| features|labelIndex|
+-----------+-----------------+----------+
Iris-setosa|[4.4,3.0,1.3,0.2]| 0.0|
Iris-setosa|[4.5,2.3,1.3,0.3]| 0.0|
Iris-setosa|[4.6,3.1,1.5,0.2]| 0.0|
Iris-setosa|[4.6,3.4,1.4,0.3]| 0.0|
Iris-setosa|[4.8,3.1,1.6,0.2]| 0.0|
+-----------+-----------------+----------+
only showing top 5 rows

Definindo o Algoritmo

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier  #biblioteca para o algoritmo da árvore de decisão
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  #utilizada para encontrar as métricas de desempenho

In [ ]:
modeloArvore = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features")  #definindo o modelo
model = modeloArvore.fit(train)  #aplicando o treinamento

In [ ]:
#realizando a previsão
predictions = model.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
prediction|labelIndex|
+----------+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+----------+----------+
only showing top 5 rows

In [ ]:
#encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [ ]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo = 0.9183673469387755

Aplicação da Floresta Randômica

In [ ]:
from pyspark.ml.classification import RandomForestClassifier  #classificador para o randomForest


In [ ]:
modeloRF = RandomForestClassifier(labelCol="labelIndex",featuresCol="features", numTrees=10)  #define o modelo
modelRF = modeloRF.fit(train)

In [ ]:
#realizando a previsão
predictions = modelRF.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
prediction|labelIndex|
+----------+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+----------+----------+
only showing top 5 rows

In [ ]:
#encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [ ]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo = 0.9183673469387755

In [ ]:
print(modelRF)

RandomForestClassificationModel (uid=RandomForestClassifier_e23ee15e7f20) with 10 trees